In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

# Build the CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, Y_test), epochs=10, batch_size=200, verbose=1)

# Evaluate the model
scores = model.evaluate(x_test, Y_test, verbose=0)
print(f"Test accuracy: {scores[1]*100:.2f}%")


2024-09-14 09:33:35.571144: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 09:33:35.613376: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 09:33:35.613794: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-14 09:33:36.402290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/10
300/300 [==============================] - 8s 25ms/step - loss: 0.4123 - accuracy: 0.8924 - val_loss: 0.1710 - val_accuracy: 0.9533
Epoch 2/10
300/300 [==============================] - 7s 23ms/step - loss: 0.1339 - accuracy: 0.9623 - val_loss: 0.0975 - val_accuracy: 0.9729
Epoch 3/10
300/300 [==============================] - 7s 23ms/step - loss: 0.0912 - accuracy: 0.9748 - val_loss: 0.0788 - val_accuracy: 0.9770
Epoch 4/10
300/300 [==============================] - 7s 23ms/step - loss: 0.0726 - accuracy: 0.9799 - val_loss: 0.0675 - val_accuracy: 0.9793
Epoch 5/10
300/300 [==============================] - 7s 23ms/step - loss: 0.0623 - accuracy: 0.9825 - val_loss: 0.0599 - val_accuracy: 0.9816
Epoch 6/10
300/300 [==============================] - 7s 23ms/step - loss: 0.0549 - accuracy: 0.9842 - val_loss: 0.0586 - val_accuracy: 0.9807
Epoch 7/10
300/300 [==============================] - 7s 23ms/step - loss: 0.0501 - accuracy: 0.9856 - val_loss: 0.0558 - val_accuracy: 0.9822

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 10)                54090     
                                                                 
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [36]:
for i in model.get_weights():
    print(i.shape)

(3, 3, 1, 32)
(32,)
(5408, 10)
(10,)


In [6]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000,)


In [4]:
import plotting
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
y_keras = model.predict(x_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))))

313/313 [==============================] - 1s 2ms/step
Accuracy: 0.9823


In [20]:
reduced_x_test = x_test[:1000]
reduced_y_test=y_test[:1000]
print(reduced_x_test.shape,reduced_y_test.shape)

(1000, 28, 28, 1) (1000,)


In [22]:
# Open the file for writing
with open('MNIST_testdata/test_dataset.c', 'w') as f:
    # Write the x_test array
    f.write("// x_test array (1000, 28, 28, 1)\n")
    f.write("float x_test[1000][28][28][1] = {\n")
    for i in range(1000):
        f.write("    {\n")
        for j in range(28):
            f.write("        {")
            for k in range(28):
                f.write("{" + f"{x_test[i][j][k][0]:.6f}" + "}, ")
            f.write("},\n")
        f.write("    },\n")
    f.write("};\n\n")

    # Write the y_test array
    f.write("// y_test array (1000,)\n")
    f.write("int y_test[1000] = {")
    for i in range(1000):
        if i % 20 == 0:
            f.write("\n    ")
        f.write(f"{y_test[i]}, ")
    f.write("\n};\n")

print("Test dataset has been saved to 'test_dataset.c'")


Test dataset has been saved to 'test_dataset.c'


In [9]:
# Function to format numpy array as C++ array
def format_array(arr, indent=0):
    if arr.ndim == 1:
        return "{" + ", ".join(f"{x:.8f}f" for x in arr) + "}"
    else:
        return "{\n" + ",\n".join(" " * (indent + 2) + format_array(sub_arr, indent + 2) for sub_arr in arr) + "\n" + " " * indent + "}"

# Generate weights.cpp file
with open("MNIST_testdata/weights.cpp", "w") as f:
    f.write("#include <vector>\n\n")

    # Conv2D layer weights and biases
    conv_weights = model.layers[0].get_weights()[0]
    conv_biases = model.layers[0].get_weights()[1]

    f.write(f"const std::vector<std::vector<std::vector<std::vector<float>>>> conv1_weights = {format_array(conv_weights)};\n\n")
    f.write(f"const std::vector<float> conv1_bias = {format_array(conv_biases)};\n\n")

    # Dense layer weights and biases
    dense_weights = model.layers[3].get_weights()[0]
    dense_biases = model.layers[3].get_weights()[1]

    f.write(f"const std::vector<std::vector<float>> dense_weights = {format_array(dense_weights)};\n\n")
    f.write(f"const std::vector<float> dense_bias = {format_array(dense_biases)};\n")

print("Weights and biases have been saved to weights.cpp")


Weights and biases have been saved to weights.cpp


In [31]:
!g++ -std=c++11 MNIST_testdata/test.cpp -o MNIST_testdata/test_nn

In [34]:
!./MNIST_testdata/test_nn

Accuracy:   95.6%
